# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/scw-comp/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Global Definitions

List of events, descriptions, prize groups, etc

In [2]:
title = '[Senior Cubers Worldwide - Weekly Comp Results](/scw-comp/results/)'
style = '<style>table {white-space: nowrap;}</style>'

prizeGroups = \
[
    ['333', '222', '333oh', 'minx'],
    ['444', '555', '666', '777'],
    ['pyram', 'skewb', 'sq1', 'clock'],
    ['333bf', '444bf', '555bf', '333fm']
]

eventPrizeGroups = {}
for groupNo in range(len(prizeGroups)):
    for event in prizeGroups[groupNo]:
        eventPrizeGroups[event] = groupNo

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [3]:
from Common_Functions import *

In [4]:
def getCorrectName(name):
    '''Handle people who regularly enter their name differently'''
    
    safeName = getSafeName(name)

    if safeName in safeNames:
        return safeNames[safeName]
    else:
        return name

In [5]:
def getIconsKey(icons):
    '''Get key based on list of icons'''

    items = []

    if '🏆' in icons:
        items.append('🏆 = overall winner')
    if '🥇' in icons:
        items.append('🥇 = 1st senior')
    if '🥈' in icons:
        items.append('🥈 = 2nd senior')
    if '🥉' in icons:
        items.append('🥉 = 3rd senior')
    if '💥' in icons:
        items.append('💥 = overall record (age group)')
    if '🔥' in icons:
        items.append('🔥 = PR average')
    if '⚡' in icons:
        items.append('⚡ = PR single')

    iconsKey = '<span style="white-space: nowrap;">' + '</span>, <span style="white-space: nowrap;">'.join(items) + '</span>'

    return iconsKey

## Generic Class

Generic class to ensure that all custom classes are printable

In [6]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Result Class

Simple class for a competition result

In [7]:
class Result(Printable):
    def __init__(self, date, rank, name, age, result, single, average, awards, solves, link):
        self.date, self.rank = date, rank
        self.name, self.age = name, age
        self.result, self.single, self.average = result, single, average
        self.awards = awards
        self.solves, self.link = solves, formatFacebookLink(link)
        
        
    def saveHeader(self, f, eventName):
        '''Save header for table'''
        
        f.write('| # | Name | Age | Single |')
        if eventName.lower() != '333fm':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(len(self.solves)):
            f.write(' Solve {} |'.format(i + 1))
        if eventName.lower() == '333fm':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | :-- | :--: |')
        if eventName.lower() != '333fm':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in self.solves:
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveResult(self, f, eventName):
        '''Save header for table'''
        
        link = '[{}](../../persons/{}/{}.md)'.format(self.name, getSafeName(self.name), eventName)
        f.write('| {:d} | {} | {} |'.format(self.rank, link, formatAge(self.age)))
        f.write(' {} |'.format(formatResult(self.single, eventName)))
        if eventName.lower() != '333fm':
            f.write(' {} |'.format(formatResult(self.average, eventName)))
        f.write(' {} |'.format(' '.join(self.awards)))
        for solve in self.solves:
            f.write(' {} |'.format(formatResult(solve, eventName)))
        if 'http' in self.link:
            f.write(' [Link]({}) |'.format(self.link))
        else:
            f.write(' |')
        f.write('\n')

## Event Class

Simple class to hold a list of results

In [8]:
class Event(Printable):
    def __init__(self, name):
        self.name = name
        self.results = []
        self.maxSolves = 0

        self.bestSingle = None
        self.bestSinglesExtra = {}
        self.bestAverage = None
        self.bestAveragesExtra = {}
        
        self.awards = {}

        self.longName = eventTitles[name][0]
        self.mediumName = eventTitles[name][1]
        self.shortName = eventTitles[name][2]


    def addResult(self, result):
        self.results.append(result)


    def isBestSingle(self, result, minAge = 40):
        isBest = False
        eventName = self.name

        if result.single:
            # Ignoring age
            if self.bestSingle:
                if self.bestSingle.single > 0:
                    if result.single > 0 and self.bestSingle.single >= result.single:
                        self.bestSingle = result
                        isBest = True
                else:
                    if result.single > 0 and self.bestSingle.single <= result.single:
                        self.bestSingle = result
                        isBest = True
            else:
                self.bestSingle = result
                isBest = True

            # Considering age
            if result.age >= minAge:
                startAge = minAge
            else:
                startAge = result.age
            for age in range(startAge, 100, 10):
                if age == result.age or (age >= minAge and age <= result.age):
                    if age in self.bestSinglesExtra:
                        if self.bestSinglesExtra[age].single > 0:
                            if result.single > 0 and self.bestSinglesExtra[age].single >= result.single:
                                self.bestSinglesExtra[age] = result
                                isBest = True
                        else:
                            if result.single > 0 and self.bestSinglesExtra[age].single <= result.single:
                                self.bestSinglesExtra[age] = result
                                isBest = True
                    else:
                        self.bestSinglesExtra[age] = result
                        isBest = True

        return isBest


    def isBestAverage(self, result, minAge = 40):
        isBest = False
        eventName = self.name

        if result.average:
            # Ignoring age
            if self.bestAverage:
                if self.bestAverage.average > 0:
                    if result.average > 0 and self.bestAverage.average >= result.average:
                        self.bestAverage = result
                        isBest = True
                else:
                    if result.average > 0 and self.bestAverage.average <= result.average:
                        self.bestAverage = result
                        isBest = True
            else:
                self.bestAverage = result
                isBest = True

            # Considering age
            if result.age >= minAge:
                startAge = minAge
            else:
                startAge = result.age
            for age in range(startAge, 100, 10):
                if age == result.age or (age >= minAge and age <= result.age):
                    if age in self.bestAveragesExtra:
                        if self.bestAveragesExtra[age].average > 0:
                            if result.average > 0 and self.bestAveragesExtra[age].average >= result.average:
                                self.bestAveragesExtra[age] = result
                                isBest = True
                        else:
                            if result.average > 0 and self.bestAveragesExtra[age].average <= result.average:
                                self.bestAveragesExtra[age] = result
                                isBest = True
                    else:
                        self.bestAveragesExtra[age] = result
                        isBest = True

        return isBest


    def getCups(self):
        cups = []
        for award in ['🏆']:
            if award in self.awards:
                cups.append('{} x {}'.format(award, self.awards[award]))

        return cups


    def getMedals(self):
        medals = []
        for award in ['🥇', '🥈', '🥉']:
            if award in self.awards:
                medals.append('{} x {}'.format(award, self.awards[award]))

        return medals


    def getAchievements(self):
        achievements = []
        for award in ['💥', '🔥', '⚡']:
            if award in self.awards:
                achievements.append('{} x {}'.format(award, self.awards[award]))

        return achievements

## Person Class

Results for a single person 

In [9]:
import unicodedata

defaultSkipDate = '9999-12-31'

class Person(Printable):
    def __init__(self, id, age, name = None, wcaId = None):
        self.id = id
        self.age = age
        self.name = name
        self.aliases = []
        self.wcaId = wcaId
        self.skipDraw = defaultSkipDate
        self.events = {}


    def isBestSingle(self, eventName, result):
        return self.events[eventName].isBestSingle(result)
    
    
    def isBestAverage(self, eventName, result):
        return self.events[eventName].isBestAverage(result)
    
    
    def addResult(self, eventName, result):
        if eventName in self.events:
            event = self.events[eventName]
        else:
            event = Event(eventName)
            self.events[eventName] = event

        event.addResult(result)

        if result.age > self.age:
            self.age = result.age
            
            
    def addAwards(self, eventName, result):
        for award in result.awards:
            if award in self.events[eventName].awards:
                self.events[eventName].awards[award] += 1
            else:
                self.events[eventName].awards[award] = 1
            
            
    def saveEventHeader(self, f, event):
        '''Save header for table'''
        
        f.write('| Date | Age | Single |')
        if event.name.lower() != '333fm':
            f.write(' Average |')
        f.write(' Awards |')
        for i in range(event.maxSolves):
            f.write(' Solve {} |'.format(i + 1))
        if event.name.lower() == '333fm':
            f.write(' Solution |\n')
        else:
            f.write(' Video |\n')

        f.write('| :--: | :--: |')
        if event.name.lower() != '333fm':
            f.write(' --: | --: |')
        else:
            f.write(' :--: |')
        f.write(' :--: |')
        for i in range(event.maxSolves):
            f.write(' --: |')
        f.write(' :-- |\n')

        
    def saveEventResult(self, f, event, result, count):
        '''Save header for table'''
        
        link = '[{}](../../results/{}/{}.md)'.format(result.date, result.date, event.name)
        f.write('| {} | {} |'.format(link, formatAge(result.age)))
        if '⚡' in result.awards:
            highlight = '**'
        else:
            highlight = ''
        f.write(' {} |'.format(formatResult(result.single, event.name, highlight)))
        if event.name.lower() != '333fm':
            if '🔥' in result.awards:
                highlight = '**'
            else:
                highlight = ''
            f.write(' {} |'.format(formatResult(result.average, event.name, highlight)))
        f.write(' {} |'.format(' '.join(result.awards)))
        for i in range(event.maxSolves):
            if len(result.solves) > i:
                if result.solves[i] > 0 and result.solves[i] == result.single and ('⚡' in result.awards):
                    highlight = '**'
                else:
                    highlight = ''
                f.write(' {} |'.format(formatResult(result.solves[i], event.name, highlight)))
            else:
                f.write(' - |')
        if 'http' in result.link:
            f.write(' [Link]({}) |'.format(result.link))
        else:
            f.write(' |')
        f.write('\n')


    def saveProfile(self):
        '''Save profile as markdown'''

        safeName = getSafeName(self.name)
        docsDir = os.path.join(projdir, 'docs', 'persons', safeName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'README.md')

        # Check for medals
        medals = False
        cups = False
        records = False
        for eventName in eventNames:
            if eventName in self.events:
                event = self.events[eventName]
                if len(event.getCups()) > 0:
                    cups = True
                if len(event.getMedals()) > 0:
                    medals = True
                if '💥' in event.awards:
                    records = True

        with open(outFile, 'w') as f:
            
            f.write('{}\n\n'.format(style))
            
            f.write('## {}\n'.format(title))
            f.write('### {}'.format(self.name))
            if self.wcaId:
                f.write(' - [{}](https://www.worldcubeassociation.org/persons/{})'.format(self.wcaId, self.wcaId))
            f.write('\n\n')

            icons = []
            if cups:
                icons.append('🏆')
            if medals and self.age >= 40:
                icons.extend(['🥇', '🥈', '🥉'])
            if records:
                icons.append('💥')
            icons.extend(['🔥', '⚡'])
            f.write('{}.\n\n'.format(getIconsKey(icons)))

            f.write('| Event | Single | Average |')
            if cups:
                f.write(' Cups |')
            if medals:
                f.write(' Medals |')
            f.write(' Achievements|\n')

            f.write('| :-- | --: | --: |')
            if cups:
                f.write(' :--: |')
            if medals:
                f.write(' :-- |')
            f.write(' :-- |\n')
            
            for eventName in eventNames:
                if eventName in self.events:
                    event = self.events[eventName]

                    f.write('| [{}]({}.md) |'.format(event.mediumName, eventName))
                    if event.bestSingle:
                        f.write(' {} |'.format(formatResult(event.bestSingle.single, eventName)))
                    else:
                        f.write(' - |')
                    if event.bestAverage:
                        f.write(' {} |'.format(formatResult(event.bestAverage.average, eventName)))
                    else:
                        f.write(' - |')
                    if cups:
                        f.write(' {} |'.format(', '.join(event.getCups())))
                    if medals:
                        f.write(' {} |'.format(', '.join(event.getMedals())))
                    f.write(' {} |\n'.format(', '.join(event.getAchievements())))

            writeGoogleSiteTag(f)

        self.saveEvents()

        
    def saveEvents(self):
        '''Save events as markdown'''

        safeName = getSafeName(self.name)
        docsDir = os.path.join(projdir, 'docs', 'persons', safeName)

        for eventName in eventNames:
            if eventName in self.events:
                outFile = os.path.join(docsDir, eventName + '.md')

                with open(outFile, 'w') as f:
                    event = self.events[eventName]

                    f.write('{}\n\n'.format(style))
                    
                    f.write('## {}\n'.format(title))
                    f.write('### [{}](README.md)'.format(self.name))
                    if self.wcaId:
                        f.write(' - [{}](https://www.worldcubeassociation.org/persons/{}?event={})'.format(
                            self.wcaId, self.wcaId, eventName))
                    f.write('\n')
                    f.write('#### {} Results'.format(event.longName))
                    f.write('\n\n')

                    icons = []
                    if '🏆' in event.awards:
                        icons.append('🏆')
                    if self.age >= 40 and len(event.getMedals()) > 0:
                        icons.extend(['🥇', '🥈', '🥉'])
                    if '💥' in event.awards:
                        icons.append('💥')
                    icons.extend(['🔥', '⚡'])
                    f.write('{}.\n\n'.format(getIconsKey(icons)))

                    event.maxSolves = 0
                    for result in event.results:
                        if len(result.solves) > event.maxSolves:
                            event.maxSolves = len(result.solves)

                    self.saveEventHeader(f, event)
                    count = len(event.results)
                    for result in reversed(event.results):
                        self.saveEventResult(f, event, result, count)
                        count -= 1

                    f.write('\n')

                    writeGoogleSiteTag(f)

## Competition Class

Results for a single competition 

In [10]:
from datetime import datetime

import os
import glob
import csv

import Levenshtein
import random

class Competition(Printable):
    def __init__(self, dataDir):
        self.dataDir = dataDir
        self.date = os.path.basename(dataDir)
        self.events = {}

        
    def saveEvent(self, eventName):
        '''Save results as markdown'''

        event = self.events[eventName]
        results = event.results
        
        docsDir = os.path.join(projdir, 'docs', 'results', self.date)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, eventName + '.md')

        with open(outFile, 'w') as f:
            
            f.write('{}\n\n'.format(style))

            f.write('## {}\n'.format(title))
            f.write('### {} {}\n\n'.format(event.longName, self.date))
            
            icons = ['🏆', '🥇', '🥈', '🥉', '💥', '🔥', '⚡']
            f.write('{}.\n\n'.format(getIconsKey(icons)))

            results[0].saveHeader(f, eventName)
            for result in results:
                result.saveResult(f, eventName)

            writeGoogleSiteTag(f)


    def saveEvents(self):
        '''Save results as markdown'''

        for eventName in self.events:
            self.saveEvent(eventName)


    def drawPrizes(self):
        '''Draw prizes from hat - quick hack for the first week'''

        prizeDraw = []
        for group in range(4):
            prizeDraw += self.prizeDraw[group]
        prizeDraw = sorted(prizeDraw)

        winners = []
        random.seed(a = self.date)
        while len(winners) < 4:
            winner = random.choice(prizeDraw)
            winners.append(winner)
            while winner in prizeDraw:
                prizeDraw.remove(winner)

        print("Prize winners {}:".format(self.date))
        names = []
        for winner in winners:
            for person in persons:
                if persons[person].id == winner:
                    names.append(person)
        print("- {}".format(", ".join(names)))


    def processSheets(self, latest):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        csvFiles = glob.glob(os.path.join(self.dataDir, '*.csv'))

        self.prizeDraw = [[], [], [], []]
        
        for fn in csvFiles:
            with open(fn, 'r') as f:
                csvReader = csv.reader(f)

                # Variable columns
                indexName = None
                indexWcaId = None
                indexAge = None
                indexBest = None
                indexAverage = None
                indexSolves = []
                indexLink = None

                # Counters, etc
                rowNo = 1
                prevResult = prevSingle = 0
                prevSeniorResult = prevSeniorSingle = 0
                rank = seniorRank = 0
                nextRank = nextSeniorRank = 1
                results = []

                eventName = os.path.basename(fn).split('.')[0]
                event = Event(eventName)
                self.events[eventName] = event

                for values in csvReader:            
                    # Process header row
                    if rowNo == 1:
                        for i in range(len(values)):
                            if values[i] == 'Name':
                                indexName = i
                            elif values[i] == 'WCA ID':
                                indexWcaId = i
                            elif values[i] == 'Age':
                                indexAge = i
                            elif values[i] == 'Best':
                                indexBest = i
                            elif values[i] == 'Average':
                                indexAverage = i
                            elif values[i] == 'Result':
                                indexBest = i
                            elif str(values[i]).startswith('Solve'):
                                indexSolves.append(i)
                            elif 'link' in str(values[i]).lower():
                                indexLink = i
        
                    # Process result row - must be after header on row 3
                    else:
                        # Fixed fields
                        name = getCorrectName(values[indexName].strip())
                        if indexWcaId:
                            wcaId = values[indexWcaId].strip()
                            if wcaId == "":
                                wcaId = None
                        else:
                            wcaId = None
                        
                        try:
                            age = interpretAge(values[indexAge])
                        except:
                            print('ERROR: Issue with age for {} in {} {}'.format(name, eventName, self.date))

                        # Retrieve single
                        if indexBest:
                            try:
                                single = numSeconds(values[indexBest])
                            except:
                                print('ERROR: Issue with best for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            single = None

                        # Retrieve average
                        if indexAverage:
                            try:
                                average = numSeconds(values[indexAverage])
                            except:
                                print('ERROR: Issue with average for {} in {} {}'.format(name, eventName, self.date))
                        else:
                            average = None

                        # Process solves
                        solves = []
                        for indexSolve in indexSolves:
                            try:
                                value = numSeconds(values[indexSolve])
                            except:
                                print('ERROR: Issue with result for {} in {} {}'.format(name, eventName, self.date))
                                raise
                                
                            solves.append(value)

                        if len(indexSolves) >= 1:
                            # Check best and auto-correct
                            tmpSingle = calculateBest(solves)
                            if single:
                                diff = round(tmpSingle - single, 2)
                                if diff != 0:
                                    print('ERROR: Best incorrect for {} in {} ({}) - calculated {}, diff {}'.format(
                                        name, event.name, self.date, formatResult(calculateBest(solves), event), diff))
                            single = tmpSingle

                            # Check average and auto-correct
                            tmpAverage = calculateAverage(solves)
                            if average:
                                diff = round(tmpAverage - average, 2)
                                if diff < -0.01 or diff > 0.01:
                                    print('ERROR: Average incorrect for {} in {} ({}) - calculated {}, diff {}'.format(
                                        name, event.name, self.date, formatResult(calculateAverage(solves), event), diff))
                            average = tmpAverage

                        # Awards will be determined later
                        awards = []
                        link = values[indexLink].strip()

                        # Determine result in seconds and rank
                        if eventName == '333fm' or eventName.endswith('bf'):
                            thisResult = single
                        else:
                            thisResult = average

                        if thisResult != prevResult or single != prevSingle:
                            rank = nextRank
                        nextRank += 1

                        if age >= 40:
                            if thisResult != prevSeniorResult or single != prevSeniorSingle:
                                seniorRank = nextSeniorRank
                            nextSeniorRank += 1

                        # Check person
                        if name in persons:
                            person = persons[name]
                            if wcaId and wcaId != person.wcaId:
                                print('Warning: Different WCA ID for {} in {} {} - {}'.format(name, eventName, self.date, wcaId))
                        else:
                            safeName = getSafeName(name)
                            if safeName in safeNames:
                                name = safeNames[safeName]
                                person = persons[name]
                            else:
                                for person in persons:
                                    if (Levenshtein.distance(person, name) < 5):
                                        personBytes = len(person)
                                        nameBytes = len(name)
                                        print("WARNING: Similar names - '{}' ({}) in {} vs existing '{}' ({})".format(
                                            name, nameBytes, eventName, person, personBytes))
                                id = len(persons) + 1
                                person = Person(id, age, name, wcaId)
                                persons[name] = person

                        # Add name to prize draw
                        prizeGroup = eventPrizeGroups[eventName]
                        if person.skipDraw > self.date and self.prizeDraw[prizeGroup].count(person.id) < 2:
                            self.prizeDraw[prizeGroup].append(person.id)
                        
                        # Report age change
                        if age > person.age and latest:
                            print('Info: Higher age for {} in {} {}'.format(name, eventName, self.date))
                        elif age < person.age and latest:
                            print('ERROR: Lower age for {} in {} {}'.format(name, eventName, self.date))

                        # Update WCA ID
                        if person.wcaId is None and wcaId is not None:
                            person.wcaId = wcaId
                        
                        # Record result
                        result = Result(self.date, rank, name, age, thisResult, single, average, awards, solves, link)
                        results.append(result)
                        person.addResult(eventName, result)
                        event.addResult(result)

                        # Check rank
                        if thisResult is None:
                            print('ERROR: No result for {} in {} {}'.format(name, eventName, self.date))
                        if (thisResult > 0 and thisResult < prevResult):
                            print('ERROR: Order is incorrect for {} in {} {}'.format(name, eventName, self.date))

                        # Determine rewards
                        if rank == 1 and single > 0:
                            result.awards.append('🏆')
                        if age >= 40 and single > 0:
                            if seniorRank == 1:
                                result.awards.append('🥇')
                            elif seniorRank == 2:
                                result.awards.append('🥈')
                            elif seniorRank == 3:
                                result.awards.append('🥉')

                        # First pass of checking for overall / age-related records and PRs
                        isRecordAverage = events[eventName].isBestAverage(result, minAge = 10)
                        isRecordSingle = events[eventName].isBestSingle(result, minAge = 10)

                        prevResult = thisResult
                        prevSingle = single

                        if age >= 40:
                            prevSeniorResult = thisResult
                            prevSeniorSingle = single

                    rowNo += 1

                # Second pass of checking for overall / age-related records and PRs
                for result in results:
                    person = persons[result.name]
        
                    if result.average and result.average > 0 and events[eventName].isBestAverage(result, minAge = 10):
                        result.awards.append('💥')
                    elif result.single > 0 and events[eventName].isBestSingle(result, minAge = 10):
                        result.awards.append('💥')

                    if result.average and result.average > 0 and person.isBestAverage(eventName, result):
                        result.awards.append('🔥')
                    if result.single > 0 and person.isBestSingle(eventName, result):
                        result.awards.append('⚡')

                    person.addAwards(eventName, result)

## Functions to Save Results

Generate markdown files for results, etc.

In [11]:
def saveProfiles():
    # Remove existing person profiles
    for i in glob.glob(os.path.join(projdir, 'docs', 'persons', '*.md')):
        os.unlink(i)

    # Remove existing person results
    for i in glob.glob(os.path.join(projdir, 'docs', 'persons', '*', '*.md')):
        os.unlink(i)

    # Save profile pages
    for person in sorted(persons):
        persons[person].saveProfile()

In [12]:
def saveCompetitors():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'events', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'persons.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('{}\n\n'.format(style))

            f.write('## {}\n'.format(title))
            f.write('### {} Competitors\n\n'.format(event.longName))

            icons = ['🏆', '🥇', '🥈', '🥉', '💥', '🔥', '⚡']
            f.write('{}.\n\n'.format(getIconsKey(icons)))

            for age in range(100, -10, -10):
                found = False

                for personName in sorted(persons):
                    person = persons[personName]
                    if eventName in person.events and age in person.events[eventName].bestSinglesExtra:

                        if found == False:
                            f.write('#### {}\n\n'.format(formatAgeLong(age)))
                            f.write('| Name | Age | Single |')
                            if eventName.lower() != '333fm':
                                f.write(' Average |')
                            f.write(' Cups | Medals | Achievements |\n')

                            f.write('| :-- |')
                            if eventName.lower() != '333fm':
                                f.write(' :--: | --: | --: |')
                            else:
                                f.write(' :--: | :--: |')
                            f.write(' :--: | :-- | :-- |\n')

                            found = True

                        link = '[{}](../../persons/{}/{}.md)'.format(person.name, getSafeName(person.name), eventName)
                        f.write('| {} |'.format(link))
                        for tmpAge in person.events[eventName].bestSinglesExtra.keys():
                            maxAge = tmpAge
                        f.write(' {} | {} |'.format(formatAge(maxAge), formatResult(
                            person.events[eventName].bestSinglesExtra[age].single, eventName)))
                        if eventName.lower() != '333fm':
                            if person.events[eventName].bestAverage:
                                f.write(' {} |'.format(formatResult(person.events[eventName].bestAveragesExtra[age].average, eventName)))
                            else:
                                f.write(' DNF |')
                        f.write(' {} |'.format(', '.join(person.events[eventName].getCups())))
                        f.write(' {} |'.format(', '.join(person.events[eventName].getMedals())))
                        f.write(' {} |\n'.format(', '.join(person.events[eventName].getAchievements())))

                if found:
                    f.write('\n')

            writeGoogleSiteTag(f)

In [13]:
def saveBestSingles():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'events', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'singles.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('{}\n\n'.format(style))

            f.write('## {}\n'.format(title))
            f.write('### {} Singles\n\n'.format(event.longName))

            icons = ['🏆', '🥇', '🥈', '🥉', '💥', '🔥', '⚡']
            f.write('{}.\n\n'.format(getIconsKey(icons)))
                    
            for age in range(100, -10, -10):
                results = []
                for personName in sorted(persons):
                    person = persons[personName]
                    if eventName in person.events:
                        event = person.events[eventName]
                        if age in event.bestSinglesExtra:
                            if event.bestSinglesExtra[age].single and event.bestSinglesExtra[age].single > 0:
                                results.append(event.bestSinglesExtra[age])

                if results:
                    f.write('#### {}\n\n'.format(formatAgeLong(age)))

                    f.write('| # | Name | Date | Age | Single | Cups | Medals | Achievements |')
                    if eventName.lower() == '333fm':
                        f.write(' Solution |\n')
                    else:
                        f.write(' Video |\n')

                    f.write('| :--: | :-- | :--: | :--: |')
                    if eventName.lower() != '333fm':
                        f.write(' --: |')
                    else:
                        f.write(' :--: |')
                    f.write(' :--: | :-- | :-- | :-- |\n')

                    rank = 1
                    prevSingle = 0
                    for result in sorted(results, key=lambda result: result.single):
                        if result.single > prevSingle:
                            f.write('| {} |'.format(rank))
                        else:
                            f.write('| |')
                        link = '[{}](../../persons/{}/{}.md)'.format(result.name, getSafeName(result.name), eventName)
                        f.write(' {} |'.format(link))
                        f.write(' [{}](../../results/{}/{}.md) |'.format(result.date, result.date, eventName))
                        f.write(' {} |'.format(formatAge(result.age)))
                        f.write(' {} |'.format(formatResult(result.single, eventName)))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getCups())))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getMedals())))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getAchievements())))

                        if 'http' in result.link:
                            f.write(' [Link]({}) |\n'.format(result.link))
                        else:
                            f.write(' |\n')

                        prevSingle = result.single
                        rank += 1

                    f.write('\n')

            writeGoogleSiteTag(f)

In [14]:
def saveBestAverages():
    for eventName in eventNames:
        count = 0

        docsDir = os.path.join(projdir, 'docs', 'events', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'averages.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('{}\n\n'.format(style))

            f.write('## {}\n'.format(title))
            f.write('### {} Averages\n\n'.format(event.longName))

            icons = ['🏆', '🥇', '🥈', '🥉', '💥', '🔥', '⚡']
            f.write('{}.\n\n'.format(getIconsKey(icons)))
                    
            for age in range(100, -10, -10):
                results = []
                for personName in sorted(persons):
                    person = persons[personName]
                    if eventName in person.events:
                        event = person.events[eventName]
                        if age in event.bestAveragesExtra:
                            if event.bestAveragesExtra[age].average and event.bestAveragesExtra[age].average > 0:
                                results.append(event.bestAveragesExtra[age])

                if results:
                    f.write('#### {}\n\n'.format(formatAgeLong(age)))
                    f.write('| # | Name | Date | Age | Average | Cups | Medals | Achievements |')
                    if eventName.lower() == '333fm':
                        f.write(' Solution |\n')
                    else:
                        f.write(' Video |\n')

                    f.write('| :--: | :-- | :--: | :--: |')
                    if eventName.lower() != '333fm':
                        f.write(' --: |')
                    else:
                        f.write(' :--: |')
                    f.write(' :--: | :-- | :-- | :-- |\n')

                    rank = 1
                    prevAverage = 0
                    for result in sorted(results, key=lambda result: result.average):
                        if result.average > prevAverage:
                            f.write('| {} |'.format(rank))
                        else:
                            f.write('| |')
                        link = '[{}](../../persons/{}/{}.md)'.format(result.name, getSafeName(result.name), eventName)
                        f.write(' {} |'.format(link))
                        f.write(' [{}](../../results/{}/{}.md) |'.format(result.date, result.date, eventName))
                        f.write(' {} |'.format(formatAge(result.age)))
                        f.write(' {} |'.format(formatResult(result.average, eventName)))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getCups())))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getMedals())))
                        f.write(' {} |'.format(', '.join(persons[result.name].events[eventName].getAchievements())))

                        if 'http' in result.link:
                            f.write(' [Link]({}) |\n'.format(result.link))
                        else:
                            f.write(' |\n')

                        prevAverage = result.average
                        rank += 1
                        count += 1

                    f.write('\n')

            if count == 0:
                f.write('Nobody has an average yet!\n')

            writeGoogleSiteTag(f)

In [15]:
def saveSeniorPodiums():
    for eventName in eventNames:
        docsDir = os.path.join(projdir, 'docs', 'events', eventName)
        if not os.path.exists(docsDir):
            os.makedirs(docsDir)

        outFile = os.path.join(docsDir, 'podiums.md')

        event = Event(eventName)

        with open(outFile, 'w') as f:

            f.write('{}\n\n'.format(style))

            f.write('## {}\n'.format(title))
            f.write('### {} Senior Podiums\n'.format(event.longName))

            icons = ['🏆', '🥇', '🥈', '🥉', '💥', '🔥', '⚡']
            f.write('{}.\n\n'.format(getIconsKey(icons)))
                    
            for competition in sorted(competitions, reverse = True):
                competition = competitions[competition]

                header = False

                if eventName in competition.events:
                    results = competition.events[eventName].results

                    for result in results:
                        pos = 0
                        if '🥇' in result.awards:
                            pos = 1
                        if '🥈' in result.awards:
                            pos = 2
                        if '🥉' in result.awards:
                            pos = 3
                        if pos > 0:
                            if not header:
                                f.write('#### [{}](../../results/{}/{}.md)\n\n'.format(competition.date, competition.date, eventName))

                                f.write('| # | Name | Age | Single |')
                                if eventName.lower() != '333fm':
                                    f.write(' Average | Awards | Video |\n')
                                else:
                                    f.write(' Awards | Solution |\n')

                                f.write('| :--: | :-- | :--: |')
                                if eventName.lower() != '333fm':
                                    f.write(' --: | --: |')
                                else:
                                    f.write(' :--: |')
                                f.write(' :--: | :-- |\n')

                                header = True

                            link = '[{}](../../persons/{}/{}.md)'.format(result.name, getSafeName(result.name), eventName)
                            f.write('| {} | {} | {} |'.format(pos, link, formatAge(result.age)))
                            f.write(' {} |'.format(formatResult(result.single, eventName)))
                            if eventName.lower() != '333fm':
                                f.write(' {} |'.format(formatResult(result.average, eventName)))
                            f.write(' {} |'.format(' '.join(result.awards)))

                            if 'http' in result.link:
                                f.write(' [Link]({}) |\n'.format(result.link))
                            else:
                                f.write(' |\n')

                    f.write('\n')

            writeGoogleSiteTag(f)

In [16]:
def saveIndexPage():
    outFile = os.path.join(projdir, 'docs', 'results', 'README.md')
    with open(outFile, 'w') as f:

        f.write('{}\n\n'.format(style))

        f.write('## {}\n'.format(title))
        f.write('### Historical Archive\n')

        f.write('#### Individuals\n')

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}](../events/{}/persons.md)'.format(event.shortName, eventName))
        f.write('Competitors - {}\n\n'.format(', '.join(eventList)))

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}](../events/{}/singles.md)'.format(event.shortName, eventName))
        f.write('Best Singles - {}\n\n'.format(', '.join(eventList)))

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            if eventName.lower() != '333fm':
                eventList.append('[{}](../events/{}/averages.md)'.format(event.shortName, eventName))
        f.write('Best Averages - {}\n\n'.format(', '.join(eventList)))

        f.write('#### Podiums\n')

        eventList = []
        for eventName in eventNames:
            event = Event(eventName)
            eventList.append('[{}](../events/{}/podiums.md)'.format(event.shortName, eventName))
        f.write('Seniors - {}\n\n'.format(', '.join(eventList)))

        f.write('#### Results\n')

        for competition in sorted(competitions, reverse = True):
            competition = competitions[competition]
            eventList = []
            for eventName in eventNames:
                if eventName in competition.events:
                    event = Event(eventName)
                    eventList.append('[{}]({}/{}.md)'.format(event.shortName, competition.date, eventName))
            f.write('{} - {}\n\n'.format(competition.date, ', '.join(eventList)))

        f.write('#### Project Code\n')

        url = 'https://github.com/Logiqx/scw-comp'
        f.write('Python code for these reports can be found at [{}]({})\n'.format(url, url))

        writeGoogleSiteTag(f)

## Save Persons

Save updated persons in JSON format

In [17]:
def savePersons():
    # Remove redundant information from persons
    for person in persons:
        if isinstance(persons[person].age, int):
            persons[person].age = formatAge(persons[person].age)

        try:
            if len(persons[person].aliases) == 0:
                del(persons[person].aliases)
        except AttributeError:
            pass

        try:
            if persons[person].skipDraw == defaultSkipDate:
                del(persons[person].skipDraw)
        except AttributeError:
            pass

        try:
            if persons[person].wcaId is None:
                del(persons[person].wcaId)
        except AttributeError:
            pass

        try:
            del(persons[person].name)
        except AttributeError:
            pass

        try:
            del(persons[person].events)
        except AttributeError:
            pass

    # Save persons
    outFile = os.path.join(projdir, 'data', 'persons.json')
    with open(outFile, 'w') as f:
        jsonDump = json.dumps(persons, default=lambda o: o.__dict__, ensure_ascii=False, indent=2, sort_keys=True)
        f.write(jsonDump)

## Main Code

Process all competitions

In [18]:
import glob
import time
import json

pc1 = time.perf_counter()

In [19]:
# Read existing persons from JSON

def as_person(d):
    if 'id' in d:
        person = Person(d['id'], interpretAge(d['age']))
        if 'wcaId' in d:
            person.wcaId = d['wcaId']
        if 'aliases' in d:
            person.aliases = d['aliases']
        if 'skipDraw' in d:
            person.skipDraw = d['skipDraw']
        return person
    else:
        return d

fn = os.path.join(projdir, 'data', 'persons.json')
if os.path.exists(fn):
    with open(fn) as f:
        jsonTxt = f.read()
        persons = json.loads(jsonTxt, object_hook = as_person)

        for person in persons:
            persons[person].name = person
else:
    persons = {}
    
safeNames = {}
for person in persons:
    safeName = getSafeName(person)
    safeNames[safeName] = person

    for alias in persons[person].aliases:
        safeName = getSafeName(alias)
        safeNames[safeName] = person

In [20]:
# Initialise events for records, etc.

events = {}
for eventName in eventNames:
    event = Event(eventName)
    events[eventName] = event
    
# Process competition data for all weeks

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'

competitions = {}

dataDirs = glob.glob(os.path.join(projdir, 'data', datePattern))
latest = False

for dataDir in dataDirs:
    if dataDir == dataDirs[-1]:
        latest = True

    competition = Competition(dataDir)
    competition.processSheets(latest)
    competition.saveEvents()
    competitions[competition.date] = competition

In [21]:
# Save results
saveCompetitors()
saveBestSingles()
saveBestAverages()
saveSeniorPodiums()
saveIndexPage()

# Save persons
saveProfiles()

# Save data
savePersons()

# Draw prizes
latestComp = sorted(competitions.keys())[-1]
competition = competitions[latestComp]
competition.drawPrizes()

Prize winners 2020-06-30:
- Pete Lee, Jason Green, Huy Nguyen, Joshua Riegel


In [22]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 6.82 seconds


## All Done!